# Processing Data

Dùng để chính thức đổi data thô thành data set, chia thành các mini batch đưa vào tập train

In [162]:
from lib_all import *

## I. Xây dựng dataset

Input:
- list_img_path: danh sách đường dẫn đến các bức ảnh
- list_anno_path: danh sách đường dẫn đến các file annotation
- phase: pha làm việc (train/val)
- transform: object của class Datatransform
- anno_xml: Để convert thông tin từ file xml ra thành một list
Output
-

In [163]:
class MyDataset(data.Dataset):
    def __init__(self, list_img_path , list_anno_path, phase, DataTransform, InfoAnno ):
        self.list_img_path = list_img_path
        self.list_anno_path = list_anno_path
        self.phase = phase
        self.DataTransform = DataTransform
        self.InfoAnno = InfoAnno

    # Tim so luong buc anh ben trong dataset
    def __len__(self):
        return len(self.list_img_path)

    # lay tung phan tu ra
    def __getitem__(self, index):
        img, ground_truth,height, width = self.pull_item(index)
        return img, ground_truth # gt la mot list 5 phan tu  thong tin lien quan den bounding box, label

    def pull_item(self, index):
        # lay thong tin ve anh
        img_path = self.list_img_path[index]
        img = cv2.imread(img_path) #BGR
        height, width, channels = img.shape

        #lay thong tin ve annotation
        anno_path = self.list_anno_path[index]
        anno_info = self.InfoAnno(anno_path, width,height)

        #tien xu ly
        img, boxes, labels = self.DataTransform(img, self.phase, anno_info[:, :4], anno_info[:, 4])

        #doi thu tu cac kieu (BGR -> RGB) (height, width, channels ->channels, height, width) de phu hop voi cac ham cua pytorch
        img = torch.from_numpy(img[:,:,(2,1,0)]).permute(2,0,1)

        # gan gia tri cua bounding box, labels vao array
        ground_truth = np.hstack((boxes, np.expand_dims(labels, axis=1)))  #stack theo chieu ngang
        return img, ground_truth, height, width


## II. Chia data set thành các mini batch
Sử dụng lại hàm DataLoader trong thư viện data ( tuy nhiên phải custom lại )
Khác với bài toán classsification thì bài toán này không thể dùng collec_fc có sẵn được mà phải customize lại do 1 img có thể có nhiều object


### custom lại collate_fn
Input:
    - mini_batch: gồm batch_size ảnh và annotation
Output:
    - Toàn bộ phải ở dạng tensor thì mới đưa được vào mạng ssd

In [164]:
def collate_fn(mini_batch):
    annotations = []
    imgs = []

    for sample in mini_batch:
        imgs.append(sample[0]) #sample[0] la thong so ve img
        annotations.append(torch.FloatTensor(sample[1]))

    # chuyen hinh dang imgs tu list[3, 300, 300]->tensor[batch_size, 3, 300, 300]
    imgs = torch.stack(imgs, dim=0)
    return imgs, annotations

 # Test

In [165]:
import nbimporter
from data.path import get_path
from data.processing_data import  DataTransform
from data.info_annotation import  InfoAnno

In [166]:
#annotation infomation
classes = ['aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow','diningtable','dog','horse','motorbike','person','pottedplant','sheep','sofa','train','tvmonitor']

#get path
root_path = './data_set/VOCdevkit/VOC2012/'
train_images_path, train_annotations_path, val_images_path, val_annotations_path = get_path(root_path)

#prepare data transform
color_mean = (104, 117, 123)
input_size = 300

transform = DataTransform(input_size, color_mean)
InfoAnno = InfoAnno(classes)
train_dataset = MyDataset(train_images_path, train_annotations_path,phase="train", DataTransform =transform,InfoAnno= InfoAnno)
val_dataset = MyDataset(val_images_path, val_annotations_path,phase="train", DataTransform =transform,InfoAnno= InfoAnno)

Chiều dài phần tử

In [167]:
print(len(train_dataset))

5717


Phần tử đầu tiên


In [168]:
print(train_dataset.__getitem__(1))

(tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]]), array([[0.47619048, 0.36115326, 0.58333333, 0.62670713, 4.        ],
       [0.68849206, 0.36115326, 0.79464286, 0.58725341, 4.        ]]))


batch_size thường để là số chẵn vì khi đó nó sẽ đưa vào sử lý xong xong trong thread được

In [169]:
train_dataloader = data.DataLoader(train_dataset, batch_size = 4,shuffle=True,collate_fn=collate_fn)

# chuyển  thành iterator
batch_iter = iter(train_dataloader) # luot qua tung phan tu
images, annotations = next(batch_iter)

Chiều dài bộ dữ liệu giảm xuống 4 lần

In [170]:
print(len(train_dataloader))

1430


Mỗi iter bao gồm 4 ảnh


In [171]:
print(images.size())

torch.Size([4, 3, 300, 300])


Mỗi iter bao gồm 4 annotation

In [172]:
print("Chieu dai annotations {}".format(len(annotations)))

Chieu dai annotations 4


In [173]:
print(annotations[0].size())

torch.Size([2, 5])
